In [21]:
import os 
import numpy as np 
import json 
import pandas as pd 
from src.funcs import UpperMPCVariablesManager, LowerMPCVariablesManager 
from src.upper_mpc import step_upper_level
from src.pumps_selector import circular_right_shift
import matplotlib.pyplot as plt 
import logging
from tqdm import tqdm 
import time 

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
data_path = '/Users/alessandroquattrociocchi/Documents/Data/pump_station_data/'
data_path_wsl = "/home/alqua/data/pump_station_data/"

In [23]:
upper_mpc_data = pd.read_parquet(os.path.join(data_path_wsl, "sim_data/sim_data_1h_full.par"))
#old_strategy_eff_power_df = pd.read_parquet(os.path.join(data_path, "sim_data/sim_data_1s_with_eff_power.parquet"))
lower_mpc_data = pd.read_csv("./data/lmpc_initialization_df.csv").set_index("time")

In [24]:
start_date = pd.to_datetime('2023-05-01 00:00:00+00:00')
date_range_higher = pd.date_range(start=start_date, periods=len(upper_mpc_data), freq='h')
date_range_lower = pd.date_range(start=start_date, periods=len(lower_mpc_data), freq='min')
upper_mpc_data.index = date_range_higher
lower_mpc_data.index = date_range_lower
upper_mpc_data["price"] = upper_mpc_data["price"]/1000 
upper_mpc_data["CO2Emission"] = upper_mpc_data["CO2Emission"]/1000
Qin_kf_est = upper_mpc_data[["inflow_kf"]].resample("1min").interpolate(method='linear').values 
start_date = upper_mpc_data.index[0]
date_range_lower = pd.date_range(start=start_date, periods=len(Qin_kf_est), freq='min')
#lower_mpc_data = lower_mpc_data.reindex(lower_mpc_data.index.union(date_range_lower), fill_value=np.nan)

with open('./data/lower_mpc_coefficients.json', 'r') as json_file:
    models_coefficients = json.load(json_file)
    
upper_mpc_data = upper_mpc_data.reset_index()

logging.basicConfig(
    filename='mpc_sim.log',
    filemode='w',  # Overwrite the log file each time the simulation restarts
    level=logging.INFO,  # Change to INFO or WARNING to reduce verbosity
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
    force=True)


In [25]:
umpc_data_buffer = UpperMPCVariablesManager()
zs = 3

In [26]:
upper_mpc_data["price"] = upper_mpc_data["price"]/1000 
upper_mpc_data["CO2Emission"] = upper_mpc_data["CO2Emission"]/1000

In [27]:
class LMPCDataBuffer:

    def __init__(self):
        self.data = {
            'p1_power': [],
            'p1_qout': [],
            'u1': [],
            'p3_qout': [], 
            'p3_power': [],
            'u3': [], 
            'p4_power': [],
            'p4_qout': [], 
            'u4': [], 
            'h': []
        }

    def initialize(self, entry_dict):
        for key in self.data:
            self.data[key].append(entry_dict.get(key, None))

    def update(self, entry_dict):
        for key in self.data:
            val = entry_dict.get(key, None)
            if isinstance(val, np.generic):
                val = val.item()
            self.data[key].append(val)

    def to_dataframe(self):
        return pd.DataFrame(self.data)

In [ ]:
/home/alqua/papers/mbs_aggregation_paper_code/mpc/sys_id/results

In [31]:
sysid_params = {"a_p1_power": -p1_power_model["den"][0], 
                "b_p1_power": p1_power_model["num"][0], 

                "a_p1_qout": -p1_qout_model["den"][0], 
                "b_p1_qout": p1_qout_model["num"][0],

                "a_p3_power": -p3_power_model["den"][0], 
                "b_p3_power": p3_power_model["num"][0], 

                "a_p3_qout": -p3_qout_model["den"][0], 
                "b_p3_qout": p3_qout_model["num"][0], 

                "a_p4_power":  -p4_power_model["den"][0], 
                "b_p4_power": p4_power_model["num"][0],

                "a_p4_qout": p4_power_model["den"][0],
                "b_p4_qout": p4_power_model["num"][0]}

NameError: name 'p1_power_model' is not defined

In [28]:
ini_dict = {
        'p1_power': 10,
        'p1_qout': 100,
        'u1': 100,
        'p3_qout': 0,
        'p3_power': 0,
        'u3':0, 
        'p4_power': 0,
        'p4_qout':0, 
        'u4':0, 
        'h': 150
    }

lmpc_data = LMPCDataBuffer()
lmpc_data.initialize(ini_dict)



In [29]:
from lower_mpc import LowerMPC

In [30]:
lower_mpc = LowerMPC(sysid_params, 
                N=60, 
                Q=1.0, 
                R=0.01)

NameError: name 'sysid_params' is not defined

In [ ]:
N = 60 
Ts = 1 
trigger = [0, 0, 1]
w1 = 30
w2 = 4
costs_per_weights = []



price_cumulative_cost = 0 
carbon_cumulative_cost = 0 
cumulative_total_cost = 0 
i = 0 
current_time = pd.to_datetime('2023-05-01 00:00:00+00:00')
logging.info("Starting MPC simulation")
print(f"Testing w1={w1}, w2={w2}")

for k in tqdm(range(0, 60*24*1)):

    if k % 60 == 0: 
    
        start_time_upper_mpc = time.perf_counter()  
        Qout, height_ref, energy, energy_cost, co2_cost, total_cost = step_upper_level(horizon=24,
                                                        prices_values=np.abs(upper_mpc_data.loc[i:i+23,"price"]),
                                                        co2_values = upper_mpc_data.loc[i:i+23,"CO2Emission"], 
                                                        inflow_values=upper_mpc_data.loc[i:i+23,"inflow_kf"],
                                                        h_init=lmpc_data_buffer.h_hist[-1], 
                                                        energy_init=lmpc_data_buffer.E_hist[:,-1:].reshape(1,3),
                                                        Qout_init=lmpc_data_buffer.Qout_hist[-1],
                                                        trigger_values=trigger, 
                                                        w1 = w1, 
                                                        w2 = w2
                                                        )
        price_cumulative_cost += energy_cost
        carbon_cumulative_cost += co2_cost
        cumulative_total_cost += total_cost

        end_time_upper_mpc = time.perf_counter()

        umpc_data_buffer.update(time=current_time,
                                Qout_hopt=Qout[0], 
                                energy_htop=energy,
                                href_hopt=height_ref[0],
                                price = np.abs(upper_mpc_data.loc[i,"price"]),
                                inflow = upper_mpc_data.loc[i,"inflow_kf"], 
                                co2_emission=upper_mpc_data.loc[i,"CO2Emission"]
                                )
        
        i += 1
        logging.info(f"Iteration {i}: Height Reference = {np.round(height_ref[0])}")
        
        
        
    print(height_ref[0])
    result = lower_mpc.step(lmpc_data.data, reference=height_ref[0], inflow=Qin_kf_est[k+zs:k+zs+N+zs]) 
    lmpc_data.update(result)
    print(result["h"])
    

start_time_lower_mpc = time.perf_counter()  
if i == 24:
    break 

NameError: name 'weights_exploration' is not defined

In [ ]:
from lower_mpc import LowerMPC

In [ ]:
root_folder = "/Users/alessandroquattrociocchi/Git/mbs_aggregation_paper/mpc/"
params_path = "sys_id/results/"

In [ ]:
p1_power_model = json.load(open(root_folder + params_path + 'tf_p1_rpm_to_p1_power.json'))
p1_qout_model = json.load(open(root_folder + params_path + 'tf_p1_rpm_to_qout_p1.json'))

p3_power_model = json.load(open(root_folder + params_path + 'tf_p3_rpm_to_p3_power.json'))
p3_qout_model = json.load(open(root_folder + params_path + 'tf_p3_rpm_to_qout_p3.json'))

p4_power_model = json.load(open(root_folder + params_path + 'tf_p4_rpm_to_p4_power.json'))
p4_qout_model = json.load(open(root_folder + params_path + 'tf_p4_rpm_to_qout_p4.json'))


In [ ]:
lmpc_data = LMPCDataBuffer()
lmpc_data.initialize(ini_dict)

In [ ]:
lmpc_data.initialize(ini_dict)

In [ ]:
for k in tqdm(range(0, 60*1)):
    
    lower_mpc = LowerMPC(sysid_params, 
                        N=60, 
                        Q=1.0, 
                        R=0.01)

    inflow = Qin_kf_est[k+zs:k+zs+N+zs]

    result = lower_mpc.step(lmpc_data.data, reference=139, inflow=inflow) 
    lmpc_data.update(result)

100%|██████████| 60/60 [00:45<00:00,  1.31it/s]
